# Introduction to the Dataset

This is an introduction to the Sentinel-2 German national forest inventory dataset generated in in the frame of the Klimba project by Max Freudenberg, Sebastian Schnell and Paul Magdon.

The dataset contains the bottom of atmosphere reflectance values for the majority of the trees in the 2012 German national forest inventory.

For questions please refer to: maximilian.freudenberg@uni-goettingen.de

## Data Loading

In [ ]:
import sqlite3
import datetime
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

The dataset comes in the form of an sqlite file. We first create an sqlite connection and tell sqlite to return strings as byte-arrays.

In [ ]:
# change this path to point to the database file
dataset_path = "/home/max/dr/extract_sentinel_pixels/datasets/S2GNFI_V1_randtime.sqlite"
conn = sqlite3.connect(dataset_path)
conn.text_factory = bytes  # this makes sqlite return strings as bytes that we can parse via numpy

To load the entire dataset, you need ~30 GB of RAM. But thanks to sqlite we can limit loading to a subset just to inspect a subset of the 
data. The data is stored in the sqlite table "data".

For demo purposes, we restrict loading to the classes spruce (10) and beech (100).

In [ ]:
df = pd.read_sql_query("SELECT * FROM data WHERE \
                       (species=10 or species=100) and present_2022=TRUE and dbh_mm>200 and height_dm>120 \
                       order by random() limit 100000", conn)
df

The bottom of atmosphere (BOA) reflectances still look strange! That's because they are displayed as raw byte characters. We can easily convert the 20 byte values to 10 16 bit signed integers:

In [ ]:
df.boa = [np.frombuffer(x, dtype=np.int16) for x in df.boa]
df

Already looks much better! But what about the time?! Let's convert it from unix time to a human readable format.

In [ ]:
df.time = [datetime.date.fromtimestamp(t) for t in df.time]
df

## Quality Filtering

As you might have noticed, there is a QAI column, which stands for quality assurance information. This information is output by the [FORCE](https://force-eo.readthedocs.io/) satellite image processing suite. The different image conditions are encoded bit-wise in a single unsigned 16 bit integer, as documented [here](https://force-eo.readthedocs.io/en/latest/howto/qai.html#quality-bits-in-force).

For convenience, here are the different bit-masks in python code:

In [ ]:
VALID        = 0b000000000000000
NODATA       = 0b000000000000001
CLOUD_BUFFER = 0b000000000000010
CLOUD_OPAQUE = 0b000000000000100
CLOUD_CIRRUS = 0b000000000000110
CLOUD_SHADOW = 0b000000000001000
SNOW         = 0b000000000010000
WATER        = 0b000000000100000
AOD_INT      = 0b000000001000000
AOD_HIGH     = 0b000000010000000
AOD_FILL     = 0b000000011000000
SUBZERO      = 0b000000100000000
SATURATION   = 0b000001000000000
SUN_LOW      = 0b000010000000000
ILLUMIN_LOW  = 0b000100000000000
ILLUMIN_POOR = 0b001000000000000
ILLUMIN_NONE = 0b001100000000000
SLOPED       = 0b010000000000000
WVP_NONE     = 0b100000000000000

Let's now look at the QAI value of the first record:

In [ ]:
df["qai"][0]

...and convert this value to binary representation:

In [ ]:
bin(10240)

Now we can count the position of the 1, starting from the right - the 1 is in position 14. Comparing this with the table above tells us, that the area is sloped. Let's check that:

In [ ]:
SLOPED

Correct!

Let's perform a simple check: Are there any pixels in our dataset that FORCE detected as water or snow? To do so, we can combine the different bit flags using the "logical or" operator "|". This compares two values and sets a bit where either a or b has a bit set.

In [ ]:
my_filter = SNOW | WATER
my_filter

In [ ]:
bin(my_filter)

We have now created a value that has two 1s in positions 5 and 6.

If we now want to find out whether some of the QAI values matches our filter, we simply have to check whether there is a 1 in the same positions as in our filter. We do this by the "logical and" operator "&" that sets a bit if the QAI value *and* our filter have a bit set in the same position. Naturally, any bit set to 1 will yield a number larger zero.

In [ ]:
mask = (df["qai"] & my_filter) > 0
mask

In [ ]:
np.sum(mask)

-> There are 1795 spruce and beech records in the subset that we have loaded where FORCE recognized water or snow. Water of course makes no sense, because the measured trees are on land, but is obvousily possible in winter.

Let's now remove all types of cloud and snow from our dataset by comparing each QAI value with a filter by the logical and operator. If the QAI values and the filter have no common bits the resulting value is zero - these are the cloud free records we are interested in.

In [ ]:
my_filter = CLOUD_BUFFER | CLOUD_CIRRUS | CLOUD_OPAQUE | CLOUD_SHADOW | SNOW
mask = (df["qai"] & my_filter) == 0
df = df[mask]

**Use logical or "|" to combine filter values, logical and "&" to compare the filter with the QAI flag and check whether the result is zero to include all valid records.**

## Plotting

Let's plot the time series of our records for beech for the Sentinel-2 band B8. You find an overview of the Sentinel-2 bands [here](https://gisgeography.com/sentinel-2-bands-combinations/).

<table><tbody><tr><th>Band</th><th>Resolution</th><th>Central Wavelength</th><th>Description</th></tr><tr><td>B1</td><td>60 m</td><td>443 nm</td><td>Ultra Blue (Coastal and Aerosol)</td></tr><tr><td>B2</td><td>10 m</td><td>490 nm</td><td>Blue</td></tr><tr><td>B3</td><td>10 m</td><td>560 nm</td><td>Green</td></tr><tr><td>B4</td><td>10 m</td><td>665 nm</td><td>Red</td></tr><tr><td>B5</td><td>20 m</td><td>705 nm</td><td>Visible and Near Infrared (VNIR)</td></tr><tr><td>B6</td><td>20 m</td><td>740 nm</td><td>Visible and Near Infrared (VNIR)</td></tr><tr><td>B7</td><td>20 m</td><td>783 nm</td><td>Visible and Near Infrared (VNIR)</td></tr><tr><td>B8</td><td>10 m</td><td>842 nm</td><td>Visible and Near Infrared (VNIR)</td></tr><tr><td>B8a</td><td>20 m</td><td>865 nm</td><td>Visible and Near Infrared (VNIR)</td></tr><tr><td>B9</td><td>60 m</td><td>940 nm</td><td>Short Wave Infrared (SWIR)</td></tr><tr><td>B10</td><td>60 m</td><td>1375 nm</td><td>Short Wave Infrared (SWIR)</td></tr><tr><td>B11</td><td>20 m</td><td>1610 nm</td><td>Short Wave Infrared (SWIR)</td></tr><tr><td>B12</td><td>20 m</td><td>2190 nm</td><td>Short Wave Infrared (SWIR)</td></tr></tbody></table>

FORCE uses all bands with 10 or 20 meters resolution. So to get Sentinel band 8 we have to select the band with index 6 in our dataset. (Python indexing starts at 0.)

In [ ]:
df["b8"] = [boa[6] for boa in df["boa"]]

Now let's group the values based on the time and species.

In [ ]:
species_groups = df[["time", "species", "b8"]].groupby("species")
grouped_df_beech = species_groups.get_group(100).groupby("time")

In [ ]:
grouped_df_beech.get_group(datetime.date(2022,3,5))

In [ ]:
averaged_series = grouped_df_beech.mean()["b8"]
averaged_series

In [ ]:
plt.plot(averaged_series)

Oh what is this? There are still negative values! We forgot to filter for NODATA values. While we're at it, let's also remove pixels with high aerosol optical depth.

In [ ]:
my_filter = NODATA | CLOUD_BUFFER | CLOUD_CIRRUS | CLOUD_OPAQUE | CLOUD_SHADOW | SNOW | AOD_HIGH | AOD_FILL | AOD_INT
mask = (df["qai"] & my_filter) == 0
df = df[mask]

In [ ]:
# let's plot it again
averaged_series = grouped_df_beech.mean()["b8"]
plt.plot(averaged_series)

Some outlier values are still present, so we'll simply remove all values below zero, because negative reflectances are unphysical. Furthermore, we can divide the BOA value by 100 to obtain the BOA reflectance in %.

In [ ]:
averaged_series = grouped_df_beech.mean().query("b8 > 0")["b8"]
averaged_series /= 100
plt.plot(averaged_series)
plt.title("Beeches, Band B8, 842nm")
plt.xlabel("Time")
plt.ylabel("BOA Reflectance [%]")

### Comparison between spruce and beech

In [ ]:
my_filter = NODATA | CLOUD_BUFFER | CLOUD_CIRRUS | CLOUD_OPAQUE | CLOUD_SHADOW | SNOW | AOD_HIGH | AOD_FILL | AOD_INT
mask = (df["qai"] & my_filter) == 0
df = df[mask]

In [ ]:
species_groups = df[["time", "species", "b8"]].groupby("species")
grouped_df_beech  = species_groups.get_group(100).groupby("time")
grouped_df_spruce = species_groups.get_group(10).groupby("time")

avg_beech = grouped_df_beech.mean().query("b8 > 0")["b8"] / 100
avg_spruce = grouped_df_spruce.mean().query("b8 > 0")["b8"] / 100

In [ ]:
plt.plot(avg_beech, label="Beech")
plt.plot(avg_spruce, label="Spruce")
plt.legend()
plt.xlabel("Time")
plt.ylabel("BOA Reflectance [%]")

We see that the data is relatively noisy. This is because we have loaded only a subset of the dataset. Loading all the data takes some time but would improve the result.

## NDVI

To explore the data even more, we can take a look at the NDVI. We define the NDVI via the red and near infrared bands (see above table). Python indexing starts at 0 and we skip all bands that have a lower resolution than 20m. We select the 842nm band, because it has 10m resolution, like the red band.

In [ ]:
def ndvi(boa):
    boa32 = boa.astype(np.float32)
    return (boa32[6] - boa32[3]) / (boa32[6] + boa32[3] + np.float32(1e-8))

In [ ]:
df["ndvi"] = df["boa"].apply(ndvi)

In [ ]:
df

In [ ]:
my_filter = NODATA | CLOUD_BUFFER | CLOUD_CIRRUS | CLOUD_OPAQUE | CLOUD_SHADOW | SNOW | AOD_HIGH | AOD_FILL | AOD_INT
mask = (df["qai"] & my_filter) == 0
df = df[mask]
ndvi_beech  = df[["time", "species", "ndvi"]].query("species == 100 and -1 < ndvi < 1").groupby("time").mean()["ndvi"]
ndvi_spruce = df[["time", "species", "ndvi"]].query("species == 10 and -1 < ndvi < 1").groupby("time").mean()["ndvi"]

In [ ]:
plt.plot(ndvi_beech, label="Beech")
plt.plot(ndvi_spruce, label="Spruce")
plt.ylim(0,1)
plt.xlabel("Time")
plt.ylabel("NDVI")
plt.legend()

## Conclusion

There are far more analyses possible with this dataset and we encourage you to try them out! Should you encounter any issues or inconsistencies with the dataset itself, don't hesitate to contact the authors.